Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
set_global_seed(0)

In [5]:
eval_env = rlcard.make('kuhn-poker', config={'seed': 0})

In [6]:
actions = ['call', 'raise', 'fold', 'check']

In [7]:
dqn_agent = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=64,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

dqn_agent.load('models/kuhn_poker_dqn/step-103978.pt')

Indices 0-2 represent the hero's hole card
* 0 - Jack
* 1 - King
* 2 - Queen

Indices 3-5 represent the number of chips the hero has bet
* 3 - 0 chips
* 4 - 1 chip
* 5 - 2 chips

Indices 6-8 represent the number of chips the villain has bet
* 6 - 0 chips
* 7 - 1 chip
* 8 - 2 chips

In [8]:
def get_obs(card, hero_bet, villain_bet):
    obs = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    card_to_index = { 'J': 0, 'Q': 1, 'K': 2 }
    obs[card_to_index[card]] = 1
    obs[hero_bet + 3] = 1
    obs[villain_bet + 6] = 1
    return obs

def get_actions(moves):
    return [actions.index(a) for a in moves]

In [9]:
# Jack, first to act or facing check
state = {
    'obs': get_obs('J', 1, 1),
    'legal_actions': [1, 2, 3]
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

In [10]:
# Queen, first to act or facing check
state = {
    'obs': get_obs('Q', 1, 1),
    'legal_actions': get_actions(['raise', 'fold', 'check'])
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

In [11]:
# King, first to act or facing check
state = {
    'obs': get_obs('K', 1, 1),
    'legal_actions': get_actions(['raise', 'fold', 'check'])
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

In [12]:
# Jack, facing bet
state = {
    'obs': get_obs('J', 1, 2),
    'legal_actions': get_actions(['call', 'fold'])
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

In [13]:
# Queen, facing bet
state = {
    'obs': get_obs('Q', 1, 2),
    'legal_actions': get_actions(['call', 'fold'])
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

In [14]:
# King, facing bet
state = {
    'obs': get_obs('K', 1, 2),
    'legal_actions': get_actions(['call', 'fold'])
}

index, _ = dqn_agent.eval_step(state)
actions[index]

'call'